# ECG Anlysis: R-Peak detection with Pan–Tompkins Algorithm
---

**Objective:** Load an ECG record from the PhysioNet I-CARE database, trim the first and last 5 minutes to avoid unstable signal segments,filter the ECG signal, detect R peaks, iterate over a list of records to filter each ECG signal, and export the time, filtered ECG, and R‑peak annotations for each record to its own CSV file.

In [ ]:
# import necessary libraries

import numpy as np
import pandas as pd
import wfdb
import scipy.signal as sig

In [2]:
# define processing parameters
DB_DIR       = "i-care/2.0/training/0427"       # PhysioNet I-CARE database directory
CHANNEL      = 0                                # ECG channel to process  
TRIM_MIN     = 5                                # minutes trimmed from start & end
FILTER_BAND  = (5.0, 15.0)                      # Hz (Pan-Tompkins band)

### Functions definition

Here we define functions that perform each of the main processing steps:

In [ ]:
def bandpass(signal, fs, low=5.0, high=15.0, order=1):
    """Butterworth band-pass filter"""
    nyq = 0.5 * fs
    b, a = sig.butter(order, [low/nyq, high/nyq], btype="band")
    return sig.filtfilt(b, a, signal)


def refine(peaks, sig_f, fs, win_ms=100):
    """±win_ms search to move each peak to the local maximum"""
    w = int((win_ms / 1000) * fs)
    return np.array([p - w + np.argmax(sig_f[max(0, p - w) : p + w]) for p in peaks], int)


def pan_tompkins_indices(sig_f, fs):
    """Pan-Tompkins R-peak detector on a pre-filtered signal """
    diff    = np.diff(sig_f, prepend=sig_f[0])
    squared = diff ** 2
    win     = int(0.150 * fs)
    integ   = np.convolve(squared, np.ones(win) / win, mode="same")
    thresh  = 0.05 * np.max(integ)
    refrac  = int(0.200 * fs)                # 200 ms
    peaks, _ = sig.find_peaks(integ, height=thresh, distance=refrac)
    return refine(peaks, sig_f, fs, 100)


def filter_and_save(record_name,
                    db_dir      = DB_DIR,
                    channel     = CHANNEL,
                    trim_min    = TRIM_MIN,
                    filter_band = FILTER_BAND):
    # 1. Load header+signal
    rec = wfdb.rdrecord(record_name, pn_dir=db_dir)
    fs  = rec.fs
    ecg = rec.p_signal[:, channel]
    t   = np.arange(rec.sig_len) / fs

    # 2. Trim off the first/last `trim_min` minutes
    skip = int(trim_min * 60 * fs)           
    ecg_trim = ecg[skip : -skip]
    t_trim   = t[skip : -skip]

    # 3. Filter and detect R peaks
    ecg_f = bandpass(ecg_trim, fs, *filter_band)
    r_idx = pan_tompkins_indices(ecg_f, fs)

    # 4. Build DataFrame
    df = pd.DataFrame({
        "time_s"  : t_trim,
        "ecg_filt": ecg_f,
        "is_R"    : False
    })
    df.loc[r_idx, "is_R"] = True

    # 5. Save to CSV file
    csv_path = f"{record_name}_filtered.csv"
    df.to_csv(csv_path, index=False)
    print(f"✓ {record_name}: {len(r_idx)} peaks → {csv_path}")

    return df

### Records processing

In [ ]:
records = ["0427_001_016_ECG","0427_002_017_ECG","0427_003_018_ECG","0427_004_019_ECG","0427_005_020_ECG","0427_006_020_ECG","0427_007_021_ECG","0427_008_022_ECG","0427_009_023_ECG","0427_010_024_ECG","0427_011_024_ECG","0427_012_025_ECG","0427_013_026_ECG","0427_014_027_ECG","0427_015_028_ECG","0427_016_028_ECG","0427_017_029_ECG","0427_018_030_ECG","0427_019_031_ECG","0427_020_032_ECG","0427_021_032_ECG","0427_022_033_ECG","0427_023_034_ECG","0427_024_035_ECG","0427_025_036_ECG","0427_026_036_ECG","0427_027_037_ECG","0427_028_038_ECG","0427_029_039_ECG","0427_030_040_ECG","0427_031_040_ECG","0427_032_041_ECG","0427_033_042_ECG","0427_034_043_ECG","0427_035_044_ECG","0427_036_044_ECG","0427_037_045_ECG","0427_038_046_ECG","0427_039_047_ECG","0427_040_048_ECG","0427_041_048_ECG","0427_042_049_ECG","0427_043_050_ECG","0427_044_051_ECG","0427_045_052_ECG","0427_046_052_ECG","0427_047_053_ECG","0427_048_054_ECG","0427_049_055_ECG","0427_050_056_ECG","0427_051_056_ECG","0427_052_057_ECG","0427_053_058_ECG","0427_054_059_ECG","0427_055_060_ECG","0427_056_060_ECG","0427_057_061_ECG","0427_058_062_ECG","0427_059_063_ECG","0427_060_064_ECG","0427_061_064_ECG","0427_062_065_ECG","0427_063_066_ECG","0427_064_067_ECG","0427_065_068_ECG","0427_066_068_ECG","0427_067_069_ECG","0427_068_070_ECG","0427_069_071_ECG","0427_070_072_ECG","0427_071_072_ECG","0427_072_073_ECG","0427_073_074_ECG","0427_074_075_ECG","0427_075_076_ECG","0427_076_076_ECG","0427_077_077_ECG","0427_078_078_ECG","0427_079_079_ECG","0427_080_080_ECG","0427_081_080_ECG","0427_082_081_ECG","0427_083_082_ECG","0427_084_083_ECG","0427_085_084_ECG","0427_086_084_ECG","0427_087_085_ECG","0427_088_086_ECG","0427_089_087_ECG","0427_090_088_ECG","0427_091_088_ECG"]

In [ ]:
for rec_name in records[3:]:
    filter_and_save(rec_name)

KeyboardInterrupt: 